# Client Examples for Red Bull Receipt OCR API

Use this notebook to test your deployed API from anywhere!


In [ ]:
# -*- coding: utf-8 -*-
import requests
import json
from pathlib import Path

# ===========================
# CONFIGURATION
# ===========================
# Replace with your actual API URL (from the deployment notebook)
API_URL = "https://xxxx-xx-xxx-xxx-xxx.ngrok-free.app"  # UPDATE THIS!

# Or use localhost if testing locally
# API_URL = "http://localhost:8000"

print("="*60)
print("🧪 TESTING RED BULL RECEIPT OCR API")
print("="*60)
print(f"API URL: {API_URL}\n")


## 1️⃣ Health Check


In [ ]:
print("1️⃣ Health Check")
print("-" * 60)

try:
    response = requests.get(f"{API_URL}/health")
    print(f"Status: {response.status_code}")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
except Exception as e:
    print(f"❌ Error: {e}")

print()


## 2️⃣ API Information


In [ ]:
print("2️⃣ API Information")
print("-" * 60)

try:
    response = requests.get(f"{API_URL}/")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
except Exception as e:
    print(f"❌ Error: {e}")

print()


## 3️⃣ Scan Receipt (Upload File)


In [ ]:
# Method 1: Upload from local file
def scan_receipt_from_file(image_path):
    """Scan receipt from local image file"""
    try:
        with open(image_path, 'rb') as f:
            files = {'file': (Path(image_path).name, f, 'image/jpeg')}
            response = requests.post(f"{API_URL}/scan", files=files)
            
        if response.status_code == 200:
            result = response.json()
            print("✅ Success!")
            print(f"\nFilename: {result['filename']}")
            print(f"\n📊 Extracted Data:")
            print(json.dumps(result['extracted_data'], indent=2, ensure_ascii=False))
            return result
        else:
            print(f"❌ Error {response.status_code}: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Example usage (update with your image path)
# result = scan_receipt_from_file("path/to/your/receipt.jpg")

print("📝 Usage:")
print('result = scan_receipt_from_file("receipt.jpg")')
print()


## 4️⃣ Scan Receipt (From URL)


In [ ]:
def scan_receipt_from_url(image_url):
    """Download image from URL and scan it"""
    try:
        # Download image
        img_response = requests.get(image_url)
        img_response.raise_for_status()
        
        # Send to API
        files = {'file': ('receipt.jpg', img_response.content, 'image/jpeg')}
        response = requests.post(f"{API_URL}/scan", files=files)
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Success!")
            print(f"\n📊 Extracted Data:")
            print(json.dumps(result['extracted_data'], indent=2, ensure_ascii=False))
            return result
        else:
            print(f"❌ Error {response.status_code}: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Example usage
# result = scan_receipt_from_url("https://example.com/receipt.jpg")

print("📝 Usage:")
print('result = scan_receipt_from_url("https://example.com/receipt.jpg")')
print()


## 5️⃣ Scan Receipt (Base64 Encoded)


In [ ]:
def scan_receipt_base64(image_path):
    """Scan receipt using base64 encoding"""
    import base64
    
    try:
        # Read and encode image
        with open(image_path, 'rb') as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')
        
        # Send to API
        payload = {"image": image_data}
        response = requests.post(
            f"{API_URL}/scan_base64",
            json=payload,
            headers={'Content-Type': 'application/json'}
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Success!")
            print(f"\n📊 Extracted Data:")
            print(json.dumps(result['extracted_data'], indent=2, ensure_ascii=False))
            return result
        else:
            print(f"❌ Error {response.status_code}: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Example usage
# result = scan_receipt_base64("receipt.jpg")

print("📝 Usage:")
print('result = scan_receipt_base64("receipt.jpg")')
print()


## 6️⃣ Batch Process Multiple Receipts


In [ ]:
def batch_scan_receipts(image_paths):
    """Scan multiple receipts"""
    results = []
    
    for i, image_path in enumerate(image_paths, 1):
        print(f"\n[{i}/{len(image_paths)}] Processing: {Path(image_path).name}")
        result = scan_receipt_from_file(image_path)
        results.append({
            'file': image_path,
            'result': result
        })
    
    return results

# Example usage
# image_files = ["receipt1.jpg", "receipt2.jpg", "receipt3.jpg"]
# results = batch_scan_receipts(image_files)

print("📝 Usage:")
print('results = batch_scan_receipts(["receipt1.jpg", "receipt2.jpg"])')
print()


## 7️⃣ Upload Image in Google Colab


In [ ]:
def scan_uploaded_image():
    """Upload and scan image in Google Colab"""
    from google.colab import files
    import io
    from PIL import Image
    import matplotlib.pyplot as plt
    
    # Upload image
    print("📤 Please upload an image file...")
    uploaded = files.upload()
    
    if not uploaded:
        print("❌ No file uploaded")
        return None
    
    # Get first uploaded file
    filename = list(uploaded.keys())[0]
    image_data = uploaded[filename]
    
    # Display image
    image = Image.open(io.BytesIO(image_data))
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Uploaded: {filename}")
    plt.show()
    
    # Scan receipt
    print(f"\n🔍 Scanning {filename}...")
    files_dict = {'file': (filename, image_data, 'image/jpeg')}
    response = requests.post(f"{API_URL}/scan", files=files_dict)
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Success!")
        print(f"\n📊 Extracted Data:")
        print(json.dumps(result['extracted_data'], indent=2, ensure_ascii=False))
        return result
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        return None

# Example usage (uncomment to use in Colab)
# result = scan_uploaded_image()

print("📝 Usage (in Google Colab):")
print('result = scan_uploaded_image()')
print()


## 8️⃣ Save Results to JSON File


In [ ]:
def save_results(results, output_file="results.json"):
    """Save extraction results to JSON file"""
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        print(f"✅ Results saved to: {output_file}")
    except Exception as e:
        print(f"❌ Error saving results: {e}")

# Example usage
# save_results(result, "receipt_data.json")

print("📝 Usage:")
print('save_results(result, "receipt_data.json")')
print()


## 💡 Complete Workflow Example


In [ ]:
# 1. Check if API is running
response = requests.get(f"{API_URL}/health")
if response.json().get("status") == "healthy":
    print("✓ API is healthy")
    
    # 2. Scan a receipt
    with open("receipt.jpg", "rb") as f:
        files = {"file": ("receipt.jpg", f, "image/jpeg")}
        result = requests.post(f"{API_URL}/scan", files=files)
    
    # 3. Process the result
    if result.status_code == 200:
        data = result.json()
        receipt_info = data["extracted_data"]
        
        # 4. Use the extracted data
        print(f"Store: {receipt_info['ten_cua_hang']}")
        print(f"Total: {receipt_info['tong_tien']}")
        print(f"Date: {receipt_info['ngay']}")
        
        # 5. Save to database or file
        save_results(data, "receipt_data.json")
else:
    print("API not healthy or unreachable.")


## Quick Start
- Update `API_URL` with your ngrok URL
- Run the examples above
- Check the interactive docs at: `{API_URL}/docs`
